In [22]:
import cv2 as cv
import numpy as np
import os

COLOR_RANGES = {
    "Red": [(np.array([0, 30, 90]), np.array([15, 255, 255])), 
            (np.array([160, 30, 90]), np.array([180, 255, 255]))],
    "Yellow": [(np.array([20, 100, 150]), np.array([30, 255, 255]))],
    "Green": [(np.array([75, 50, 100]), np.array([95, 255, 255]))]
}

# 创建颜色掩码函数
def create_color_mask(img_hsv, color_name):
    ranges = COLOR_RANGES[color_name]
    mask = None
    for lower, upper in ranges:
        current_mask = cv.inRange(img_hsv, lower, upper)
        mask = current_mask if mask is None else cv.bitwise_or(mask, current_mask)
    return mask

# 预处理交通灯的函数
def preprocess_traffic_light(image):
    # 转换为HSV颜色空间
    img_hsv = cv.cvtColor(image, cv.COLOR_BGR2HSV)
    
    # 保存最高亮度的圆形信息
    best_circle = None
    best_color = None
    max_brightness = -1  # 用于保存最高亮度

    # 遍历红、黄、绿三个颜色
    for color_name in COLOR_RANGES.keys():
        # 创建颜色掩码
        mask = create_color_mask(img_hsv, color_name)
        
        # 对原图进行掩码操作
        masked_img = cv.bitwise_and(image, image, mask=mask)
        
        # 转换为灰度图
        gray = cv.cvtColor(masked_img, cv.COLOR_BGR2GRAY)
        
        # 使用 Hough Circle Transform 查找圆
        circles = cv.HoughCircles(
            gray, 
            cv.HOUGH_GRADIENT, 
            dp=1.2, 
            minDist=30, 
            param1=50, 
            param2=30, 
            minRadius=5, 
            maxRadius=50
        )
        
        # 如果检测到了圆形，逐个处理
        if circles is not None:
            circles = np.round(circles[0, :]).astype("int")

            for (x, y, r) in circles:
                # 获取圆的亮度，使用 V 通道 (亮度)
                mask_circle = np.zeros_like(gray)
                cv.circle(mask_circle, (x, y), r, 255, -1)  # 创建圆形掩码
                
                # 检查掩码中是否有有效像素
                if np.sum(mask_circle[y-r:y+r, x-r:x+r]) == 0:
                    continue  # 如果没有有效像素，跳过该圆形
                
                # 计算亮度平均值
                brightness = np.mean(img_hsv[y-r:y+r, x-r:x+r, 2][mask_circle[y-r:y+r, x-r:x+r] > 0])
                
                # 更新最高亮度圆的信息
                if brightness > max_brightness:
                    max_brightness = brightness
                    best_circle = (x, y, r)
                    best_color = color_name
    
    # 如果找到了亮度最高的圆
    if best_circle is not None:
        return best_color
    
    # 如果没有找到圆形，根据颜色区域的占比进行判断
    color_area = {}
    for color_name in COLOR_RANGES.keys():
        mask = create_color_mask(img_hsv, color_name)
        color_area[color_name] = np.sum(mask > 0)  # 计算每种颜色的掩码像素总数
    
    # 返回占比最多的颜色
    return max(color_area, key=color_area.get)

# 遍历文件夹中的图片并检测红灯、黄灯、绿灯
def process_image_folder(folder_path):
    image_files = sorted(os.listdir(folder_path))
    
    for image_file in image_files:
        image_path = os.path.join(folder_path, image_file)
        image = cv.imread(image_path)
        
        if image is None:
            print(f"无法读取图片: {image_file}")
            continue
        
        # 检测交通灯颜色
        detected_color = preprocess_traffic_light(image)
        print(f"{image_file}: Detected Traffic Light Color: {detected_color}")

# 调用函数处理文件夹
image_folder = "tlights"  # 更新为你的图片文件夹路径
process_image_folder(image_folder)


100_01.jpg: Detected Traffic Light Color: Red
100_02.jpg: Detected Traffic Light Color: Red
100_03.jpg: Detected Traffic Light Color: Green
100_04.jpg: Detected Traffic Light Color: Green
100_05.jpg: Detected Traffic Light Color: Green
100_06.jpg: Detected Traffic Light Color: Green
100_07.jpg: Detected Traffic Light Color: Red
100_08.jpg: Detected Traffic Light Color: Red
100_09.jpg: Detected Traffic Light Color: Green
100_10.jpg: Detected Traffic Light Color: Green
100_11.jpg: Detected Traffic Light Color: Green
100_12.jpg: Detected Traffic Light Color: Red
100_13.jpg: Detected Traffic Light Color: Green
100_14.jpg: Detected Traffic Light Color: Green
100_15.jpg: Detected Traffic Light Color: Red
100_16.jpg: Detected Traffic Light Color: Green
100_17.jpg: Detected Traffic Light Color: Green
100_18.jpg: Detected Traffic Light Color: Green
100_19.jpg: Detected Traffic Light Color: Red
100_20.jpg: Detected Traffic Light Color: Green
100_21.jpg: Detected Traffic Light Color: Green
100_22